# Puente entre Mundos: Traducción de Tokens (BERT ↔ GPT-4)

### Traducción de Tokens

BERT y GPT-4 tienen diccionarios totalmente diferentes.
* El token `101` para BERT es `[CLS]`.
* El token `101` para GPT-4 es algo totalmente distinto (probablemente una sílaba rara).

Para "traducir" de uno a otro, no podemos mapear número a número. Debemos usar el **Texto** como puente universal:
1.  **Decodificar** los tokens de origen a Texto (String).
2.  **Codificar** ese texto con el tokenizador de destino.

In [1]:
!pip install tiktoken
import tiktoken
from transformers import BertTokenizer

# Cargamos ambos tokenizadores
bertTokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
gpt4Tokenizer = tiktoken.get_encoding('cl100k_base')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Ejercicio 1: Funciones de Traducción
Aquí definimos las funciones que actúan como puente.
**Nota Clave:** Cuando venimos de BERT, usamos `skip_special_tokens=True`. Si no lo hacemos, el texto resultante tendría `[CLS]` y `[SEP]` escritos literalmente, y GPT-4 los tokenizaría como corchetes y letras, ensuciando la traducción.

In [2]:
# Exercise 1: Write translation functions

# translation functions
def bert2gpt4(bertToks):
  # 1. Decodificar BERT a texto limpio (quitando [CLS] y [SEP])
  b = bertTokenizer.decode(bertToks, skip_special_tokens=True)
  # 2. Codificar ese texto a formato GPT-4
  g = gpt4Tokenizer.encode(b)
  return g

def gpt42bert(gpt4Toks):
  # 1. Decodificar GPT-4 a texto
  g = gpt4Tokenizer.decode(gpt4Toks)
  # 2. Codificar ese texto a formato BERT
  # Nota: bertTokenizer.encode añade automáticamente nuevos [CLS] y [SEP]
  b = bertTokenizer.encode(g)
  return b


# just checking that it gives no errors
text = 'I wish chocolate were purple.'
print(bert2gpt4(bertTokenizer.encode(text)))
print(gpt42bert(gpt4Tokenizer.encode(text)))

[72, 6562, 18414, 1051, 25977, 13]
[101, 1045, 4299, 7967, 2020, 6379, 1012, 102]


### Ejercicio 2: De BERT a GPT-4 y vuelta a BERT
Vamos a ver si la traducción es perfecta o si se pierde información en el camino.
Fíjate en los tokens especiales al final.

In [3]:
# Exercise 2: BERT --> GPT4 --> BERT

# sample text
text = "I wanted to paste in a thought-provoking quote here, but I didn't."
print(f'Original text\n  {text}\n')

# initial encoding (BERT)
bertTox = bertTokenizer.encode(text)
print(f'BERT tokens:\n  {bertTox}\n')

# translate to GPT4 (Usando nuestra función)
b2g = bert2gpt4(bertTox)
# Imprimimos qué texto ve GPT-4 (debería ser el texto limpio)
print(f'BERT to GPT4 (decoded content):\n  {gpt4Tokenizer.decode(b2g)}\n')

# back-translate to BERT (Vuelta al origen)
back2bert = gpt42bert(b2g)
print(f'Back to BERT:\n  {bertTokenizer.decode(back2bert)}')

Original text
  I wanted to paste in a thought-provoking quote here, but I didn't.

BERT tokens:
  [101, 1045, 2359, 2000, 19351, 1999, 1037, 2245, 1011, 4013, 22776, 14686, 2182, 1010, 2021, 1045, 2134, 1005, 1056, 1012, 102]

BERT to GPT4 (decoded content):
  i wanted to paste in a thought - provoking quote here, but i didn't.

Back to BERT:
  [CLS] i wanted to paste in a thought - provoking quote here, but i didn't. [SEP]


### Ejercicio 3: GPT-4 a BERT y vuelta a GPT-4
Ahora el camino inverso. GPT-4 es más eficiente (usa menos tokens). Al pasarlo por BERT y volver, veremos cómo cambia la representación numérica.

In [4]:
# Exercise 3: GPT4 --> BERT --> GPT4

# sample text
text = "I still don't have a good quote here. Now it's too late."
print(f'Original text\n  {text}\n')

# initial encoding (GPT-4)
gptTox = gpt4Tokenizer.encode(text)
print(f'GPT-4 tokens:\n  {gptTox}\n')

# translate to BERT
g2b = gpt42bert(gptTox)
print(f'Translated to BERT tokens:\n  {g2b}\n')
print(f'BERT sees: "{bertTokenizer.decode(g2b)}"\n')

# back-translate to GPT4
back2gpt = bert2gpt4(g2b)

print(f'Back to GPT-4 tokens:\n  {back2gpt}')
print(f'Final text: "{gpt4Tokenizer.decode(back2gpt)}"')

Original text
  I still don't have a good quote here. Now it's too late.

GPT-4 tokens:
  [40, 2103, 1541, 956, 617, 264, 1695, 12929, 1618, 13, 4800, 433, 596, 2288, 3389, 13]

Translated to BERT tokens:
  [101, 1045, 2145, 2123, 1005, 1056, 2031, 1037, 2204, 14686, 2182, 1012, 2085, 2009, 1005, 1055, 2205, 2397, 1012, 102]

BERT sees: "[CLS] i still don't have a good quote here. now it's too late. [SEP]"

Back to GPT-4 tokens:
  [72, 2103, 1541, 956, 617, 264, 1695, 12929, 1618, 13, 1457, 433, 596, 2288, 3389, 13]
Final text: "i still don't have a good quote here. now it's too late."


### Explicación del Ejercicio 3: El "Teléfono Roto" (GPT-4 → BERT → GPT-4)

En este ejercicio hacemos el viaje de ida y vuelta, pero empezando desde GPT-4. Aquí descubriremos algo fascinante sobre la "pérdida de información".

#### 1. Codificación Inicial (GPT-4)
Empezamos con una frase normal: *"I still don't have..."*
GPT-4 es "Case Sensitive" (distingue mayúsculas), así que el token para **"I"** es distinto al de **"i"**.

#### 2. Traducción a BERT (`gpt42bert`)
Usamos nuestra función puente. Aquí ocurre el cambio drástico:
* El texto se convierte a números de BERT.
* **Se agregan los especiales:** Aparece `101` al inicio y `102` al final.
* **¡OJO! Pérdida de Mayúsculas:** Como estamos usando `bert-base-uncased`, al pasar por este tokenizador, **todas las mayúsculas se destruyen**. La "I" se convierte en "i".

#### 3. Regreso a GPT-4 (`bert2gpt4`)
Tomamos los tokens de BERT y los convertimos de vuelta.
* La función decodifica el texto (que ahora está todo en minúsculas: *"i still don't..."*).
* Al volver a codificar esto con GPT-4, **los números serán diferentes al principio**.
* ¿Por qué? Porque para GPT-4, la palabra "I"